In [1]:
%matplotlib inline
import numpy as np
import xarray as xr
import cubedsphere as cs

In [2]:
Vres_list = [20,40,80,160]
Hres_list = [48,96,192,384]
eps = 1e-10
scaling = 1e-9 # make area smaller

In [4]:
# warning: this cell takes very long to run!

for i,Hres in enumerate(Hres_list):
    for j,Vres in enumerate(Vres_list):
        print('loading',Hres,Vres,end=' ')
        datadir = ("../../output/C{0}/C{0}L{1}/".format(Hres,Vres))
        ds = cs.open_FV3data(datadir,"tracer_daily")
        dr = ds['plume01'].load()
        
        dr.values[dr.values<eps] = eps # correct negative values
        
        print('calculating...',end=' ')
        # higher-Vres layers are thiner
        dr_entro = (-dr*np.log(dr)*dr['area']*scaling/Vres).sum(dim=['tile','pfull','y','x'])
        
        print('writting...',end='\n')
        dr_entro.to_netcdf('plume01_entropy_C{0}L{1}.nc'.format(Hres,Vres))

loading 48 20 calculating... writting...
loading 48 40 calculating... writting...
loading 48 80 calculating... writting...
loading 48 160 calculating... writting...
loading 96 20 calculating... writting...
loading 96 40 calculating... writting...
loading 96 80 calculating... writting...
loading 96 160 calculating... writting...
loading 192 20 calculating... writting...
loading 192 40 calculating... writting...
loading 192 80 calculating... writting...
loading 192 160 calculating... writting...
loading 384 20 calculating... writting...
loading 384 40 calculating... writting...
loading 384 80 calculating... writting...
loading 384 160 calculating... writting...


## initial entropy

In [64]:
ds_init = cs.open_FV3data('../../rundirs/C48/INPUT/',"fv_tracer.res",restart=True)

In [65]:
dr_init = ds_init['plume06']
dr_init_entro = (-dr_init*np.log(dr_init)*dr_init['area']*scaling/20).sum(dim=['tile','pfull','y','x'])
dr_init_entro['time'] += 7.0
dr_init_entro

<xarray.DataArray (time: 1)>
dask.array<sum-aggregate, shape=(1,), dtype=float64, chunksize=(1,)>
Coordinates:
  * time     (time) float64 8.0

In [66]:
dr_temp =  xr.open_dataarray(maindir+'plume01_entropy_C{0}L{1}'.format(48,20),
                             decode_times=False)
xr.concat([dr_init_entro,dr_temp],'time')

<xarray.DataArray (time: 9)>
dask.array<concatenate, shape=(9,), dtype=float64, chunksize=(1,)>
Coordinates:
  * time     (time) float64 8.0 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0

## Merge all files

In [67]:
maindir = "~/FV3/output/processed/entropy/"

ds_HsVs_arr = [] # list of timeseries at different horizontal and vertical resolutions

for Hres in Hres_list:
    ds_Vs_arr = [] # list of timeseries at vertical different resolutions
    for Vres in Vres_list:
        
        dr_temp =  xr.open_dataarray(maindir+'plume01_entropy_C{0}L{1}.nc'.format(Hres,Vres),
                                     decode_times=False)
        dr_temp = xr.concat([dr_init_entro,dr_temp],'time')
        
        ds_Vs_arr.append( dr_temp.copy()) 
            
        ds_Vs = xr.concat(ds_Vs_arr, dim='Vres')
        ds_Vs["Vres"]=np.array(Vres_list)
        
    ds_HsVs_arr.append( ds_Vs )

ds_HsVs = xr.concat(ds_HsVs_arr, dim='Hres')
ds_HsVs["Hres"]=np.array(Hres_list)

### creat new file

In [68]:
ds_HsVs.to_netcdf(maindir+'plume01_entropy_all.nc')